In [27]:
import pandas as pd
import numpy as np

In [28]:
data1 = pd.read_excel("extract2.xlsx")
data1.drop(index=0, inplace=True)
data1.drop(columns=['PLY_NO', 'PYG_GST', 'APR_FLG', 'BSN_USE', 'BLD_CLM_CNT'], inplace=True)   # dropping cols with same values throughout
data1.reset_index(drop=True, inplace=True)
data1

,PLY_PRD,CST_OCP,TIR_PRM_AM,UCP_DY,UCP_RSN,OWS_DTL,SUM_INU,PRM_AM,PRM_WTH_IPT,RSK_PRM_INC_CRD_SCR,...,TIR_RSK_PRM,ITM_LMT_VL,MAN_PRT_PRM,ACD_PRM,OBL_PRM,PRV_ISN_BLD,PRV_ISN_CNE,PPT_ELG,RSK_PRM_OVR_TAX_SCG,CTS_CLM_CNT
0,DC1,RETI,0,30,Pendingsale(deceased),OR,NaN,NaN,NaN,NaN,...,0,550000,NaN,NaN,NaN,9,9,1,15.68,NaN
1,DC1,OTHR,195.2,30,NaN,OR,500000,174.29,195.2,162.41,...,159.51,99999999,NaN,NaN,NaN,5,5,1,34.8,0
2,DC1,OTHR,NaN,30,NaN,OR,0,39.6,NaN,NaN,...,NaN,99999999,1.789269,0.515735,0,7,7,1,77.99,0
3,DC1,OTHR,0,30,Pendingsale(deceased),UR,0,0,0,0,...,0,100000,NaN,NaN,NaN,0,2,1,27.13,0
4,DC1,RETI,0,30,NaN,OR,0,0,0,NaN,...,0,75000,0.688533,0.111698,0.068097,99,9,1,24.39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,DC1,RETI,0,30,NaN,OR,0,0,0,NaN,...,0,1000000,4.24824,0.3502,0.33187,99,4,1,31.32,0
72,DC1,OTHR,0,30,Pendingsale(deceased),FR,NaN,NaN,NaN,NaN,...,38.92,100000000,NaN,NaN,NaN,0,0,1,57.67,0
73,DC1,RETI,0,30,NaN,OR,NaN,NaN,NaN,NaN,...,0,100000,NaN,NaN,NaN,99,99,NaN,41.34,0
74,DC1,RETI,27.59,30,NaN,OR,2000,24.63,27.59,NaN,...,11.7,5000,9.31995,0.39542,0,99,0,NaN,27.06,0


In [29]:
print("Column, Missing Values, Unique Values")
for col in data1.columns:
    print(f"{col}, {data1[col].isna().sum()}, {data1[col].nunique()}")

Column, Missing Values, Unique Values
PLY_PRD, 0, 2
CST_OCP, 20, 3
TIR_PRM_AM, 8, 24
UCP_DY, 18, 2
UCP_RSN, 42, 1
OWS_DTL, 18, 4
SUM_INU, 37, 10
PRM_AM, 37, 17
PRM_WTH_IPT, 40, 15
RSK_PRM_INC_CRD_SCR, 64, 8
RSK_PRM_NO_CRD_SCR, 64, 8
CVR_IND, 10, 2
CVR_TYP_LMT, 60, 6
CVR_SEL, 45, 2
TIR_LMT_VL, 25, 11
TIR_RSK_PRM, 3, 30
ITM_LMT_VL, 3, 20
MAN_PRT_PRM, 42, 31
ACD_PRM, 42, 26
OBL_PRM, 42, 22
PRV_ISN_BLD, 6, 11
PRV_ISN_CNE, 6, 11
PPT_ELG, 25, 1
RSK_PRM_OVR_TAX_SCG, 0, 76
CTS_CLM_CNT, 35, 2


In [30]:
categorical_cols = ["CST_OCP", "UCP_DY", "OWS_DTL", "CVR_IND", "CVR_SEL", "CTS_CLM_CNT"]

# Replace string indicators of missingness with NaN
data1.replace(["", "NA", "None"], np.nan, inplace=True)

for col in categorical_cols:
    if col in data1.columns:
        choices = data1[col].dropna().unique()
        data1[col] = data1[col].apply(lambda x: np.random.choice(choices) if pd.isna(x) else x)

# standard_fill_cols
data1["UCP_RSN"] = data1["UCP_RSN"].fillna("Unknown")
data1["PPT_ELG"] = data1["PPT_ELG"].fillna(0)

C:\Users\niksh\AppData\Local\Temp\ipykernel_20352\3220750870.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data1.replace(["", "NA", "None"], np.nan, inplace=True)


In [31]:
categorical_cols.append("UCP_RSN")
categorical_cols.append("PPT_ELG")
categorical_cols.append("PLY_PRD")

rem_cols = data1.columns.difference(categorical_cols).tolist()

from scipy.stats import norm

for col in rem_cols:
    if col in data1.columns:
        non_null_values = data1[col].dropna().values
        if len(non_null_values) == 0:
            continue

        # Fit a normal distribution
        mu, sigma = norm.fit(non_null_values)

        n_missing = data1[col].isna().sum()
        # Generate samples
        sampled_values = np.random.normal(mu, sigma, n_missing)

        # Ensure no negatives — resample those that are < 0
        while np.any(sampled_values < 0):
            neg_idx = sampled_values < 0
            sampled_values[neg_idx] = np.random.normal(mu, sigma, np.sum(neg_idx))

        data1.loc[data1[col].isna(), col] = sampled_values

In [35]:
data1.to_csv("cleaned_extract2.csv", index=False)

In [32]:
data2 = pd.read_excel("fraud.xlsx")
data2 = data2.dropna(subset=['CLAIM_TYPE_DESCRIPTION']).reset_index(drop=True)
data2

,REGISTERED_DATE,LOSS_DATE,CLAIM_TYPE_DESCRIPTION,INVEST_DATE,Closed Date,SAVED_VALUE_ESTIMATE,ACTION_STATUS_DESCRIPTION,PRODUCT_CODE,REASON,Rule Outcome Code,ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME
0,2023-04-17,2023-04-17,Escape of Water,2023-04-20,2023-05-12,3000.0,Suspect,HAS,Fraud Suspected - Exaggerated,S003,2023-04-19 12:00:10.447
1,2023-04-17,2023-04-16,Theft,2023-04-17,2023-04-17,0.0,Clear after Investigation,HAA,Null,S003,2023-04-17 15:33:45.303
2,2023-04-17,2023-02-18,Theft,2023-04-20,2023-04-25,45190.0,Clear after Investigation,DC1,Null,S002,2023-04-20 09:31:44.437
3,2023-04-17,2023-04-17,Accidental damage Option,2023-04-18,2023-05-25,343.0,Suspect,DC1,Fraud Suspected - Claim Paid,S001,2023-04-17 16:15:32.263
4,2023-04-17,2023-04-15,Accidental damage Option,2023-04-25,2023-10-03,3000.0,Suspect,DC1,Fraud Suspected - Exaggerated,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...
9355,2025-07-28,2025-07-20,Escape of Water,2025-08-02,2025-08-05,0.0,Clear after Investigation,DC2,Null,S001,2025-07-28 13:14:30.467
9356,2025-07-28,2025-07-28,Fire,2025-07-31,2025-07-31,0.0,Clear after Investigation,DC2,Null,S001,2025-07-28 15:59:23.347
9357,2025-08-01,2025-08-01,Fire,2025-08-07,2025-08-07,0.0,Clear after Investigation,DC2,Null,S002,2025-08-01 12:22:51.573
9358,2025-08-01,2025-07-31,Accidental loss (EXC),2025-08-08,2025-08-13,0.0,Clear after Investigation,DC2,Null,S002,2025-08-01 12:50:01.883


In [33]:
print("Column, Missing Values, Unique Values")
for col in data2.columns:
    print(f"{col}, {data2[col].isna().sum()}, {data2[col].nunique()}")

Column, Missing Values, Unique Values
REGISTERED_DATE, 0, 798
LOSS_DATE, 0, 925
CLAIM_TYPE_DESCRIPTION, 0, 19
INVEST_DATE, 0, 675
Closed Date, 0, 655
SAVED_VALUE_ESTIMATE, 0, 1193
ACTION_STATUS_DESCRIPTION, 0, 3
PRODUCT_CODE, 0, 8
REASON, 0, 24
Rule Outcome Code, 89, 3
ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME, 89, 9271


In [34]:
col = "Rule Outcome Code"
choices = data2[col].dropna().unique()
data2[col] = data2[col].apply(lambda x: np.random.choice(choices) if pd.isna(x) else x)

# Fill NA in ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME with a random date between REGISTERED_DATE and REGISTERED_DATE + 4 days, using normal distribution
mask = data2['ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME'].isna()
register_dates = data2.loc[mask, 'REGISTERED_DATE']

# Generate random offsets (in days) from a normal distribution centered at 2, std=1, clipped to [0, 4]
offsets = np.clip(np.random.normal(loc=2, scale=1, size=mask.sum()), 0, 4)
random_dates = register_dates + pd.to_timedelta(offsets, unit='D')

data2.loc[mask, 'ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME'] = random_dates
data2

,REGISTERED_DATE,LOSS_DATE,CLAIM_TYPE_DESCRIPTION,INVEST_DATE,Closed Date,SAVED_VALUE_ESTIMATE,ACTION_STATUS_DESCRIPTION,PRODUCT_CODE,REASON,Rule Outcome Code,ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME
0,2023-04-17,2023-04-17,Escape of Water,2023-04-20,2023-05-12,3000.0,Suspect,HAS,Fraud Suspected - Exaggerated,S003,2023-04-19 12:00:10.447000000
1,2023-04-17,2023-04-16,Theft,2023-04-17,2023-04-17,0.0,Clear after Investigation,HAA,Null,S003,2023-04-17 15:33:45.303000000
2,2023-04-17,2023-02-18,Theft,2023-04-20,2023-04-25,45190.0,Clear after Investigation,DC1,Null,S002,2023-04-20 09:31:44.437000000
3,2023-04-17,2023-04-17,Accidental damage Option,2023-04-18,2023-05-25,343.0,Suspect,DC1,Fraud Suspected - Claim Paid,S001,2023-04-17 16:15:32.263000000
4,2023-04-17,2023-04-15,Accidental damage Option,2023-04-25,2023-10-03,3000.0,Suspect,DC1,Fraud Suspected - Exaggerated,S003,2023-04-19 14:36:43.570258989
...,...,...,...,...,...,...,...,...,...,...,...
9355,2025-07-28,2025-07-20,Escape of Water,2025-08-02,2025-08-05,0.0,Clear after Investigation,DC2,Null,S001,2025-07-28 13:14:30.467000000
9356,2025-07-28,2025-07-28,Fire,2025-07-31,2025-07-31,0.0,Clear after Investigation,DC2,Null,S001,2025-07-28 15:59:23.347000000
9357,2025-08-01,2025-08-01,Fire,2025-08-07,2025-08-07,0.0,Clear after Investigation,DC2,Null,S002,2025-08-01 12:22:51.573000000
9358,2025-08-01,2025-07-31,Accidental loss (EXC),2025-08-08,2025-08-13,0.0,Clear after Investigation,DC2,Null,S002,2025-08-01 12:50:01.883000000


In [36]:
data2.to_csv("cleaned_fraud.csv", index=False)

In [ ]:
from scipy.stats import gaussian_kde

def scale_dataframe(data, target_rows, categorical_cols, numeric_cols, random_state=None):
    """
    Scale up a mixed-type DataFrame to target_rows by synthetic resampling 
    while preserving per-column data distributions.
    """
    np.random.seed(random_state)
    new_data = pd.DataFrame()

    # --- Handle categorical columns ---
    for col in categorical_cols:
        if col not in data.columns:
            continue
        col_data = data[col]
        # Compute probability of each category
        probs = col_data.value_counts(normalize=True)
        # Sample categories according to observed frequency
        sampled = np.random.choice(probs.index, size=target_rows, p=probs.values)
        new_data[col] = sampled

    # --- Handle numeric columns ---
    for col in numeric_cols:
        if col not in data.columns:
            continue
        col_data = data[col].values
        # Fit Kernel Density Estimation (KDE)
        kde = gaussian_kde(col_data)
        # Generate synthetic samples
        sampled = kde.resample(target_rows).reshape(-1)
        # Clip to observed range (optional, ensures realism)
        sampled = np.clip(sampled, col_data.min(), col_data.max())
        new_data[col] = sampled

    # Preserve column order as in the original DataFrame
    new_data = new_data[data.columns]

    return new_data.reset_index(drop=True)

# Generate scaled data
scaled_data1 = scale_dataframe(data1, target_rows=15000,
                              categorical_cols=categorical_cols,
                              numeric_cols=rem_cols,
                              random_state=42)

scaled_data1.to_csv("scaled_extract2.csv", index=False)

In [ ]:
def scale_dataframe2(
    data,
    target_rows,
    date_cols,
    categorical_cols,
    numeric_cols,
    random_state=None
):
    """
    Rule-based synthetic scaling of a mixed-type dataframe with:
    - Date columns (sampled within range)
    - Categorical columns (sampled by frequency)
    - Numeric columns (KDE resampling)
    """

    if random_state is not None:
        np.random.seed(random_state)

    new_data = pd.DataFrame()

    # --- 1. DATE COLUMNS -----------------------------------------------------
    for col in date_cols:
        if col not in data.columns:
            continue
        
        # Convert to datetime (if not already)
        col_data = pd.to_datetime(data[col])

        # Range of observed dates
        min_date = col_data.min()
        max_date = col_data.max()

        # Sample random dates within observed range
        total_days = (max_date - min_date).days
        rand_days = np.random.randint(0, total_days + 1, size=target_rows)
        sampled_dates = min_date + pd.to_timedelta(rand_days, unit="D")

        new_data[col] = sampled_dates.astype(str)

    # --- 2. CATEGORICAL COLUMNS --------------------------------------------
    for col in categorical_cols:
        if col not in data.columns:
            continue

        col_data = data[col]
        probs = col_data.value_counts(normalize=True)

        sampled = np.random.choice(
            probs.index,
            size=target_rows,
            p=probs.values
        )
        new_data[col] = sampled

    # --- 3. NUMERIC COLUMNS --------------------------------------------------
    for col in numeric_cols:
        if col not in data.columns:
            continue
        
        col_data = data[col].astype(float).values

        # KDE Fit
        kde = gaussian_kde(col_data)

        sampled_vals = kde.resample(target_rows).reshape(-1)

        # Clip to keep realism
        sampled_vals = np.clip(sampled_vals, col_data.min(), col_data.max())

        new_data[col] = sampled_vals

    # Match column ordering with original dataframe
    new_data = new_data[data.columns]

    return new_data.reset_index(drop=True)

date_cols = [
    "REGISTERED_DATE",
    "LOSS_DATE",
    "INVEST_DATE",
    "Closed Date",
    "ENQUIRY_REALTIME_RESULT_CREATE_DATE_TIME"
]

categorical_cols = [
    "CLAIM_TYPE_DESCRIPTION",
    "ACTION_STATUS_DESCRIPTION",
    "PRODUCT_CODE",
    "REASON",
    "Rule Outcome Code"
]

numeric_cols = ["SAVED_VALUE_ESTIMATE"]

scaled_data2 = scale_dataframe2(
    data=data2,
    target_rows=15000,
    date_cols=date_cols,
    categorical_cols=categorical_cols,
    numeric_cols=numeric_cols,
    random_state=42
)

scaled_data2.to_csv("scaled_fraud.csv", index=False)

In [ ]:
scaled_data1 = scaled_data1.reset_index(drop=True)
scaled_data2 = scaled_data2.reset_index(drop=True)

data3 = pd.concat([scaled_data1, scaled_data2], axis=1)
data3

In [49]:
data3.to_csv("scaled_combined.csv", index=False)